In [8]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import issparse


In [9]:
def read_alist_file(filename):
    """
    Read an alist file and return the parity-check matrix H
    as a SciPy sparse matrix (COO format).
    """

    with open(filename, 'r') as f:
        # First line: N, M
        N, M = map(int, f.readline().split())

        # Second line: max_sum_dv, max_sum_dc
        max_sum_dv, max_sum_dc = map(int, f.readline().split())

        # Variable-node degrees (length N)
        sum_dv = list(map(int, f.readline().split()))

        # Check-node degrees (length M)
        sum_dc = list(map(int, f.readline().split()))

        # Read variable-node adjacency lists
        data = []
        for _ in range(N):
            row = list(map(int, f.readline().split()))
            data.append(row)

    # Convert to NumPy array: shape (N, max_sum_dv)
    row_idx = np.array(data)

    # MATLAB is 1-based; convert to 0-based indexing
    row_idx = row_idx - 1

    # Column indices: repeated variable indices
    col_idx = np.repeat(np.arange(N), max_sum_dv)

    # Flatten row indices row-wise
    row_idx = row_idx.flatten()

    # Keep only valid entries (row_idx >= 0)
    mask = row_idx >= 0
    row_idx = row_idx[mask]
    col_idx = col_idx[mask]

    # Values are all ones
    values = np.ones(len(row_idx), dtype=int)

    # Build sparse matrix (M x N)
    H = coo_matrix((values, (row_idx, col_idx)), shape=(M, N))

    return H


#### example

In [19]:
def write_alist_file(H, filename, overwrite=False):
    """
    Write a parity-check matrix H (M x N) to an alist file.

    H: SciPy sparse matrix recommended (CSR/CSC/COO all ok)
    """
    import os

    if os.path.exists(filename) and not overwrite:
        print(f"Warning file: {filename} exists! Not overwriting.")
        return

    if not issparse(H):
        # Allow dense input too, but convert
        H = csr_matrix(H)

    M, N = H.shape

    # Use CSC for column adjacency, CSR for row adjacency
    Hc = H.tocsc()
    Hr = H.tocsr()

    # Degrees
    sum_dv = np.diff(Hc.indptr).astype(int)  # length N
    sum_dc = np.diff(Hr.indptr).astype(int)  # length M

    max_sum_dv = int(sum_dv.max()) if N > 0 else 0
    max_sum_dc = int(sum_dc.max()) if M > 0 else 0

    # --- Write header ---
    with open(filename, "w") as f:
        f.write(f"{N} {M}\n")
        f.write(f"{max_sum_dv} {max_sum_dc}\n")
        f.write(" ".join(map(str, sum_dv.tolist())) + "\n")
        f.write(" ".join(map(str, sum_dc.tolist())) + "\n")

        # --- Variable-node adjacency lists (N lines, each max_sum_dv ints) ---
        # Each line lists check-node indices (rows) connected to that variable (column), 1-based, padded with 0.
        for j in range(N):
            rows = Hc.indices[Hc.indptr[j]:Hc.indptr[j+1]]
            rows = np.sort(rows) + 1  # 1-based; sort for canonical output
            if max_sum_dv > 0:
                out = np.zeros(max_sum_dv, dtype=int)
                out[:rows.size] = rows
                f.write(" ".join(map(str, out.tolist())) + "\n")
            else:
                f.write("\n")

        # --- Check-node adjacency lists (M lines, each max_sum_dc ints) ---
        # Each line lists variable-node indices (cols) connected to that check (row), 1-based, padded with 0.
        for i in range(M):
            cols = Hr.indices[Hr.indptr[i]:Hr.indptr[i+1]]
            cols = np.sort(cols) + 1  # 1-based; sort for canonical output
            if max_sum_dc > 0:
                out = np.zeros(max_sum_dc, dtype=int)
                out[:cols.size] = cols
                f.write(" ".join(map(str, out.tolist())) + "\n")
            else:
                f.write("\n")


In [20]:
H = read_alist_file("test.alist").tocsr()
print(H)
write_alist_file(H, "test_out.alist", overwrite=True)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 152 stored elements and shape (8, 52)>
  Coords	Values
  (0, 0)	1
  (0, 3)	1
  (0, 4)	1
  (0, 7)	1
  (0, 10)	1
  (0, 12)	1
  (0, 19)	1
  (0, 21)	1
  (0, 22)	1
  (0, 25)	1
  (0, 27)	1
  (0, 31)	1
  (0, 33)	1
  (0, 36)	1
  (0, 39)	1
  (0, 41)	1
  (0, 43)	1
  (0, 45)	1
  (0, 46)	1
  (1, 1)	1
  (1, 2)	1
  (1, 5)	1
  (1, 6)	1
  (1, 11)	1
  (1, 13)	1
  :	:
  (6, 34)	1
  (6, 36)	1
  (6, 40)	1
  (6, 42)	1
  (6, 45)	1
  (6, 50)	1
  (7, 0)	1
  (7, 2)	1
  (7, 7)	1
  (7, 9)	1
  (7, 13)	1
  (7, 14)	1
  (7, 16)	1
  (7, 21)	1
  (7, 23)	1
  (7, 25)	1
  (7, 26)	1
  (7, 28)	1
  (7, 33)	1
  (7, 35)	1
  (7, 37)	1
  (7, 41)	1
  (7, 43)	1
  (7, 44)	1
  (7, 51)	1


In [18]:
H = read_alist_file("test.alist").tocsr()
write_alist_file(H, "test_out.alist", overwrite=True)

H2 = read_alist_file("test_out.alist").tocsr()
print((H != H2).nnz)  # should be 0

210
